# Лабораторная работа: Капрекар и неподвижные точки функции К(i)
* * * * * * * * * * * *
### Вычитание в соответствующей системе счисления
На вход поступает два массива массивов list и основание системы счисления

In [ ]:
bad = ['A', 'B', 'C', 'D', 'E', 'F']

def substr(a, b, num_sys):
    res = []
    for i in range(len(b)):
        if (b[i] - a[i] >= 0): res.append(b[i] - a[i])
        else: 
            res.append(b[i] + num_sys - a[i])
            k = i + 1
            while ((b[k] == 0) and (k < len(b))):
                b[k] = b[k] + num_sys - 1
                k = k + 1
            #if (k == len(b):  - число < 0, в рамках наших вычислений такого не случится... можно добавить исключение
            b[k] = b[k] - 1
    res.reverse()
    ## для единственности найденных чисел сразу отсортируем его
    res.sort(reverse = True)
    for i in range(len(res)): 
        if(res[i] > 9): res[i] = bad[res[i] - 10]
    return ("".join(map(str,res)))

### Функция Капрекар. 
На вход поступает - вектор выборки, количество разрядов числа и система счисления

In [ ]:
def kaprekar(number, digit, num_sys):
    digits = [int(d, num_sys) for d in number]
    #while(digit > len(elem)):
    #    digits.append('0')
    digits.sort()
    min_sq = digits
    min_num = int("".join(map(str,digits)))
    digits.sort(reverse=True)
    max_sq = digits
    max_num = int("".join(map(str,digits)))
    return substr(max_sq, min_sq, num_sys)

#### Введем переменные:  
list magicNumber содержит информацию о всех найденных неподвижных точках.  
            [  ( значение точки или цикл точек, разрядность найденного значения, система счисления значения точки, длина до него (максимальная))  ]  
  
list usedNumber вспомогательный, чтобы найти цикл.

In [ ]:
import math
magicNumber = {0: (math.inf, 2)}
usedNumber = []

### Построение всех различных (для функции) чисел

Как будем строить число? Начнем с малого, заполним набор всех чисел для случая 2-разрядного числа с системе счисления по основанию 2. Это числа "11" и "10". Как далее происходит переход к большей разрядности или основанию С.С.?  

#### Разрядность:  
11 10       ->  (111, 110) + (100)  
111 110 100 -> (1111, 1110) + (1100) + (1000)   

#### Система счисления:  
11 10       + 22 21 20  
111 110 100 + 222 221 220 211 210 200


In [ ]:
import numpy as np
batch_size = 10
# начальный массив возможных значений для d = 2, r = 2
start = ["11", "10"]

# строим таким образом, чтобы были только max значения (можно min или премешанное в виде 51423, 
# главное, чтобы для одному набору цифр соответствовала только одна комбинация из них)

def add_d(arr, digit):
    tmp = []
    for i in arr:
        for k in range(3):
            if (int(i[digit - 2]) < k): break
            else: tmp.append(i + str(k))
    return tmp

def add_r(rez, number, digit, r):
    if (r > 10): number = number + bad[r - 11]
    else: number = number + (str(r - 1))
    for i in range (r):
        if (digit <= 1):
            rez.append(number)
            return rez
        else:
            add_r(rez, number, digit - 1, r - i)
    return rez



In [ ]:
for d in range(2,17):
    if (d > 2): start = add_d(start, d)
    for r in range(2,17):
        if (r > 2): 
            collection.extend(add_r([],"", d, r))
        else: collection = start.copy()
        print ("d = ",d,"r = ",r)
        # count - счетчик длины до стационарной точки
        count = 0
        # 100 раз берем рандомные числа из collection и находим магические числа или циклы
        for n in range (101):
            order = np.random.permutation(len(collection))[0 : min(d * 5, batch_size)]
            x = np.array(collection)[order]
            for number in x:
                usedNumber.clear()
                prev = "0"
                while True:
                    prev = number
                    usedNumber.append(number)
                    number = kaprekar(number, d, r) 
                    if (int(number,r) > 0):
                        print ("number is" + number)
                    count = count + 1
                    if (int(number,r) == 0):
                        break
                    if ((number == prev) and (magicNumber.get(number) == None)):
                        magicNumber[number] = (r, count)
                        print ("число " + number + " является неподвижной точкой: d = " + str(d) +" r = " + str(r) + "\n")
                        break
                    else:
                        if(number in usedNumber): #найден цикл
                            print("цикл")
                            print(usedNumber)
                            break
            if (len(order) < batch_size):
                break

In [ ]:
magicNumber["23"] = (3, 5, 7)
magicNumber

In [ ]:
print(magicNumber.get("23"))

In [ ]:
kaprekar("10000",2,2)

In [ ]:
b10000
b00001